In [ ]:
import pandas as pd
import numpy as np
import requests
import tweepy
import os
import json
import time
import re
import matplotlib.pyplot as plt
import warnings

Gather Data

In [ ]:
#Gathering or Importing the twitter-archive-enhanced.csv file into a DataFrame.
archive_1 = pd.read_csv('twitter-archive-enhanced.csv')
archive_1.head()

In [ ]:
# Download .tsv file from the website provided by Udacity
url="https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
response = requests.get(url)

with open('image-predictions.tsv', 'wb') as file:
    file.write(response.content)

# Read in tsv file as a Pandas DataFrame    
image_predictions = pd.read_csv('image-predictions.tsv', sep='\t')

In [ ]:
consumer_key = '.....'
consumer_secret = '.....'
access_token = '.....'
access_secret = '.....'

In [ ]:
# Variables created for tweepy query
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [ ]:
# For loop that will add each available tweet to a new line of tweet_json.txt
with open('tweet-json.txt', 'a', encoding='utf8') as f:
    for tweet_id in archive_1 ['tweet_id']:
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet._json, f)
            f.write('\n')
        except:
            continue

In [ ]:
# Append each tweet into a list
tweets_data = []

tweet_file = open('tweet-json.txt', "r")

for line in tweet_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue
        
tweet_file.close()

In [ ]:
# Create tweet_info DataFrame
tweet_info = pd.DataFrame()

In [ ]:
# Add below variables to tweet_info DataFrame
tweet_info['id'] = list(map(lambda tweet: tweet['id'], tweets_data))
tweet_info['retweet_count'] = list(map(lambda tweet: tweet['retweet_count'], tweets_data))
tweet_info['favorite_count'] = list(map(lambda tweet: tweet['favorite_count'], tweets_data))

Assess

In [ ]:
# Assessing Data
archive_1.head(10)

In [ ]:
archive_1.info()

In [ ]:
# View last few rows of twitter_archive DataFrame
archive_1.tail()

In [ ]:
# View descriptive statistics of twitter_archive DataFrame
archive_1.describe()

In [ ]:
# View first few rows of image_predictions DataFrame
image_predictions.head()

In [ ]:
# View last rows of image_predictions DataFrame
image_predictions.tail()

In [ ]:
# Review general information of image_predictions DataFrame
image_predictions.info()

In [ ]:
# Breif view of descriptive statistics of image_predictions DataFrame
image_predictions.describe()

In [ ]:
tweet_info.head()

In [ ]:
tweet_info.tail()

In [ ]:
# Review information of tweet_info DataFrame
tweet_info.info()

In [ ]:
# View descriptive statistics of tweet_information DataFrame
tweet_info.describe()

In [ ]:
# Rows in twitter_archive that contain '&amp;' in place of '&' in 'text' column
archive_1[archive_1.text.str.contains('&amp;')]

In [ ]:
# View number of entries for each source
archive_1.source.value_counts()

In [ ]:
# View rows where the value of 'name' is lowercase - indicates that it is not an actual name
archive_1.loc[(archive_1['name'].str.islower())]

In [ ]:
archive_1.loc[(archive_1['name'].str.islower()) & (archive_1['text'].str.contains('name is'))]

In [ ]:
archive_1[archive_1.text.str.contains(r"(\d+\.\d*\/\d+)")]

Quality 
* A lot of Columns are empty values. Most retweeted status columns. 
* In archive_1 'Name' column there is high volume entry for "a" which can not be labeled as name. 
* Also nulls represented as 'None' in columns 'name', 'doggo', 'floofer', 'pupper','puppo'. Addition to that the timestamp column is an object which has to be a datetime object 
* Missing Dog Names 
* Some unknown Characters included like ""&amp" 
* Incorrect Dog Names 
* Erroneous datatype (tweet_id, in_reply_to_status, Source)
* Missing values like no value assigned in some cells under the name column

Tidiness 
* To tide up this data I am going to do some clean up. The first step is removing retweetes together with related columns like retweeted_status_id, 'retweeted_status_user_id' and 'retweeted_status_timestamp 
* Column name "expanded_url" not explicit enough in twitter data frame

Clean

In [ ]:
# Create copies of original DataFrames that way we work on the copy for our analysis
archive_1_clean = archive_1.copy()
image_predictions_clean = image_predictions.copy()
tweet_info_clean = tweet_info.copy()

Define
Create variable of columns that are no longer needed


Code

In [ ]:
# Create variable of columns that are no longer needed and drop them from the DataFrame 
columns = ['doggo', 'floofer', 'pupper', 'puppo']
archive_1_clean = archive_1_clean.drop(columns, axis=1)

Test

In [ ]:
archive_1_clean.head(5)

Define

Combine tweet_info and image_predicitions to archive_1 table

Code

In [ ]:
archive_1_clean = pd.merge(left=archive_1_clean, right=tweet_info_clean, left_on='tweet_id', right_on='id', how='inner')

In [ ]:
archive_1_clean = archive_1_clean.merge(image_predictions_clean, on='tweet_id', how='inner')

In [ ]:
archive_1_clean = archive_1_clean.drop('id', axis=1)

In [ ]:
archive_1_clean = archive_1_clean[archive_1_clean['retweeted_status_id'].isnull()]

Define
Remove unused Columns

Code

In [ ]:
# Deleting or Removing columns we wont be using
archive_1_clean['in_reply_to_status_id'].value_counts()

Test

In [ ]:
archive_1_clean.columns

Analyze

In [ ]:
time_df = archive_1_clean[['timestamp', 'retweet_count', 'favorite_count', 'rating_numerator', 'rating_denominator']].copy()

In [ ]:
time_df.set_index('timestamp', inplace=True)

In [ ]:
time_df['rating_ratio'] = time_df['rating_numerator']/time_df['rating_denominator']

* Update columns variable and drop columns related to retweets

Code

In [ ]:
columns = ['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp']
archive_1_clean = archive_1_clean.drop(columns, axis=1)

In [ ]:
#In order to pull ratings with images, we can isolate the entry without images
archive_1_clean = archive_1_clean.dropna(subset=['expanded_urls'])

In [ ]:
sum(archive_1_clean['expanded_urls'].isnull())

In [ ]:
#Replace "NaN" to "N/A" in name column
archive_1_clean['name'] = archive_1_clean['name'].replace('N/A', np.NaN)

In [ ]:
# Set the index to be the timestamp so time is displayed properly in plots
time_df.set_index('timestamp', inplace=True)

In [ ]:
time_df['rating_ratio'] = time_df['rating_numerator']/time_df['rating_denominator']

In [ ]:
pd.set_option('display.max_colwidth', -1)

Test

In [ ]:
archive_1_clean.head()

In [ ]:
archive_1.rating_numerator.value_counts()

In [ ]:
archive_1.rating_denominator.value_counts()

In [ ]:
#Remove tweets without any rating displayed
archive_1_clean = archive_1_clean[archive_1_clean.tweet_id != 835152434251116546]

In [ ]:
archive_1_clean[archive_1_clean.tweet_id == 835152434251116546]

In [ ]:
archive_1_clean['timestamp'] = archive_1_clean['timestamp'].str.replace('+', '')

In [ ]:
#save the file to csv file
archive_1_clean.to_csv('archive_1_master.csv')

In [ ]:
time_df = archive_1_clean[['timestamp', 'rating_numerator', 'rating_denominator']].copy()

In [ ]:
# Set the index to be the timestamp so time is displayed properly in plots
time_df.set_index('timestamp', inplace=True)

In [ ]:
# Create rating_ration variable by dividing the rating numerator by the deonominator to normalize scores which are not 
# out of 10
time_df['rating_ratio'] = time_df['rating_numerator']/time_df['rating_denominator']

In [ ]:
# Gather data from twitter API using Python's Tweepy library 
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer


In [ ]:
tweet_ids = archive_2.tweet_id.values
len(tweet_ids)

In [ ]:
count = 0
fails_dict = {}
start = timer()


In [ ]:
with open('tweet-json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet_info = api.get_status(tweet_id, tweet_mode='extended')
            retweet_count=tweet_info.retweet_count
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("TweepError")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

Change type of data to category

Code

In [ ]:
archive_1_clean['source'] = archive_1_clean['source'].astype('category')

Test

In [ ]:
archive_1_clean.source.value_counts()

Store

In [ ]:
# Save clean DataFrame to csv file
archive_1_clean.to_csv('archive_1_master.csv')

In [ ]:
archive_1_master = pd.read_csv('archive_1_master.csv')
archive_1_master.info()

Analyze

In [ ]:
time_df = archive_1_clean[['timestamp', 'rating_numerator', 'rating_denominator']].copy()

In [ ]:
time_df['Ratio'] = time_df['rating_numerator']/time_df['rating_denominator']

In [ ]:
plt.hist(archive_1_master['rating_numerator'], range = (2, 14));
plt.xlabel('Rating Numerator')
plt.ylabel('Frequency')
plt.title('Frequency Ratio')

In [ ]:
archive_1_master[['rating_numerator', 'rating_denominator']].plot(style = '.', alpha = 0.8)
plt.title('Numerator and Denominator')
plt.xlabel('Numerator')
plt.ylabel('Denominator');


In [ ]:
archive_1_clean.name

In [ ]:
archive_1_clean["predicted_breed"]